In [ ]:
#==========================#
# Import relevant packages #
#==========================#


import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.special import erfinv
from iminuit import Minuit
from scipy.stats import chi2
from scipy.stats import binom
%matplotlib inline
from scipy.stats import poisson

# Report 1 - Estimating Parameters with MC Generated Samples

## Description

The report based around a particle decay of $X \rightarrow D$, and a parameter related to the matter/anti-matter asymmetry of the Universe will be measured.

The PDF of the relevant particle decay is described by:
$$
P(t;\tau,\Delta m_s, V) \propto (1+V\sin{(\Delta mt)}) \times \exp{-\frac{t}{\tau}}
$$
where

* $t$ is the observable quantity - the decay time of each decay.
* $\tau$ is a lifetime parameter.
* $\Delta m$ is a mass difference parameter which leads to sinusoidal oscillations superimposed on the exponential decay.
* $V$ is a parameter which measures matter/anti-matter asymmetry and has the value zero if
the universe is symmetric.

And the nominal value of the parameters are:

* $\tau = 1.5$
* $\Delta m = 20.0$
* $V = 0.1$

We will first plot the PDF to see the distribution within the range 0 to 10. At this stage, something else in addition is to vary the parameters a little to see how change of each parameter affects our PDF distribution.

In [ ]:
#==================================================#
# Create a function returning the PDF distribution #
#==================================================#

def pdf_distribution(t, tau, delta_m, V):
    return ( (1 + V*np.sin(delta_m*t))*np.exp(-t/tau) )

In [ ]:
#=====================================================#
# Define parameters for plotting the PDF distribution #
#=====================================================#

loBound = 0.
hiBound = 10.
n_interval = 1000
t = np.linspace(loBound, hiBound, n_interval, endpoint=True)
# nominal values for the parameters
tau1 = 1.5
delta_m1 = 20.
v1 = 0.1
# Three more sets of parameters to see how our PDF distribution change
tau2 = 3.
delta_m2 = 10.
v2 = 1.

tau3 = 2.
delta_m3 = 10.
v3 = 4.

In [ ]:
#===========================#
# Plot the PDF distribution #
#===========================#

plt.plot(t, pdf_distribution(t, tau1, delta_m1, v1), ls='-.', color='r', label='Nominal Parameters Values')
plt.plot(t, pdf_distribution(t, tau2, delta_m2, v2), ls='--', color='g', label='Set 1 Varied Parameters Values')
plt.plot(t, pdf_distribution(t, tau3, delta_m3, v3), color='orange', label='Set 2 Varied Parameters Values')
plt.title(r'Probability Density Function for the Particle Decay of $X \rightarrow D$')
plt.xlabel('Time (A.U.)')
plt.ylabel(r'$P(t;\tau,\Delta m, V)$ (A.U.)')
plt.legend()
plt.show()

The above plot shows the distribution of our PDF that characterizes the particle decay, and three of them have different parameters, this is to show how the parameters change the waveform of our PDF distribution.

As seen from above, and equation from the top description section. It appears that the function is decaying in a sinusoidal way. And the parameters ($\tau, \Delta m, V$) represents  the decay constant, the angular frequency and the amplitude, respectively.

## Question 1

In this question, we'll use toy Monte Carlo event generation to simulate multiple pseudo-experiments, and use this method to determine the expected statistical precision with which one could measure each of the parameters with 10000 events.

The first step here is to create a PDF class which generates random points based on the PDF function given in the description for the relevant particle decay, and also contain methods/functions in setting different parameters.

In [ ]:
#==============================================================================================#
# Create a PDF class used to model the oscillatory exponential decay pdf function as described #
#==============================================================================================#

class DecayPDF(object):

    # Constructor
    def __init__(self, loBound, hiBound, tau, delta_m, V):
        self.loBound = loBound
        self.hiBound = hiBound
        self.tau = tau
        self.delta_m = delta_m
        self.V = V
    
        # initialize distribution mass array
        self.mass = []

    
    #———————————————————————————————————————————#
    # Function used to find maximum of function #
    #———————————————————————————————————————————#

    def find_max(self):
        # First generate a grid of x points
        x = np.linspace(self.loBound, self.hiBound, endpoint=True, num=100000)
        # Next, evaluate the function at all the x, and return the maximum
        y = self.evaluate(x)
        return y.max()

    
    #———————————————————————————————————————————#
    # Function used to find minimum of function #
    #———————————————————————————————————————————#

    def find_min(self):
        # First generate a grid of x points
        x = np.linspace(self.loBound, self.hiBound, endpoint=True, num=100000)
        # Next, evaluate the function at all the x, and return the minimum
        y = self.evaluate(x)
        return y.min()


    #————————————————————————————————————————————————#
    # Function used to set passed vars as parameters #
    #————————————————————————————————————————————————#

    def setParameters(self, tau, delta_m, V):
        self.tau = tau
        self.delta_m = delta_m
        self.V = V


    #————————————————————————————————————————————————#
    # Evaluate the PDF and return un-normalized vals #
    #————————————————————————————————————————————————#

    def evaluate(self, t):
        return ( (1+self.V*np.sin(self.delta_m*t))*np.exp(-t/self.tau) )

    
    #—————————————————————————————————————————————————————————————————————————————————————————————#
    # From the PDF distribution for our decay, sample random points (here using the 'box' method, #
    # which this technique uses the generation of two random numbers, starting from anuniform     #
    # random number generator. Consider the algorithm equivalent to generating random numbers     #
    # in a “box”. Then only keeping those which fall under the curve.                             #
    #—————————————————————————————————————————————————————————————————————————————————————————————#

    def next(self):
        doLoop = True
        while (doLoop):
            x = np.random.uniform(self.loBound, self.hiBound)
            y1 = self.evaluate(x)
            y2 = np.random.uniform(self.find_min(), self.find_max())
            # now consider the acceptance if y1 is above x-axis
            if (y2<y1):
                filtered_x = x
                self.mass.append(x)
                return filtered_x


    #———————————————————————————————————————————#
    # Function which returns the mass attribute #
    #———————————————————————————————————————————#

    def returnMass(self):
        return np.array(self.mass)


    #——————————————————————————————————————#
    # Function which clears the mass array #
    #——————————————————————————————————————#

    def flushMass(self):
        self.mass.clear()


    #————————————————————————————————————————————————#
    # Evaluate the integral of the pdf within limits #
    #————————————————————————————————————————————————#

    def integrate(self, loBound, highBound):
        integral_result, integral_err = quad(self.evaluate, loBound, highBound)
        return integral_result

Now let's first generate a toy dataset with the PDF class we wrote in the cell above, and make a histogram for this dataset to test that our PDF class is running and working.

In [ ]:
# Nominal parameter values
tau_nom = 1.5
delta_m_nom = 20.
V_nom = 0.1

# Initialize the PDF class and feed in the nominal parameters
pdf = DecayPDF(loBound, hiBound, tau_nom, delta_m_nom, V_nom)

# Generate 10000 events for this first test toy dataset
nEvents = 10000
for i in range(nEvents):
    pdf.next()

# Return the mass array and flush them so that it resets and the class can be used again for generating new toy datasets
evts = pdf.returnMass()
pdf.flushMass()

In [ ]:
# plot our first generated dataset with our pdf class
plt.hist(evts, 150, color='orange', label='Generated Data')
plt.title('Histogram for the First Generated Dataset Using the PDF Class')
plt.xlabel('Time (A.U.)')
plt.ylabel('Event Entries')
plt.legend()
plt.show()

Looking at the above, our first generated dataset appears to be sensible, following the general trend of the pdf function as we expect. Next, we would like to use a maximum likelihood fit to determine the precision expected for each of the parameters $\tau$,$\Delta m$ and $V$. In order to do this, 100 generated toy datasets need to be fitted.

First, we will write a maximum likelihood fit class for this, using the negative of the log of the joint likelihood. Then we will fit using the class below on the 1 generated dataset above to see whether our class works well or not.


In [ ]:
#==========================================================================================#
# Create class for negative log-likelihood minimization statistic used for fitting the pdf #
#==========================================================================================#

class NegativeLLcalculator(object):
    
    def __init__(self, pdf, data):
        self.pdf = pdf
        self.data = data


    # function used to update data, mainly for the ability to recycle this class for different fit pdfs
    def updateData(self, data):
        self.data = data

    # Function used to calculate log likelihood
    def calc_neg_LL(self):
        likelihood = self.pdf.evaluate(self.data)/(self.pdf.integrate(self.pdf.loBound, self.pdf.hiBound))
        log_likelihood = np.log(likelihood)
        return -log_likelihood.sum()

    # To calcualte an NLL for our pdf
    def evaluate(self, tau, delta_m, V):
        nll = 0.
        self.pdf.setParameters(tau, delta_m, V)
        # here, compute the log likelyhood
        return self.calc_neg_LL()

Now we will use iMinuit to fit our dataset with the minimization statistics written above.

In [ ]:
# Define fit parameters in a dictionary
init_params = {
        'tau':                  1.5,
        'delta_m':              20.,
        'V':                    0.1,
}

# First initialize the pdf classes for function we use to fit, and the fitting statistics
pdf = DecayPDF(loBound, hiBound, **init_params)
fit_stats = NegativeLLcalculator(pdf, evts)

# Initialize iminuit minimizer
minimizer = Minuit(fit_stats.evaluate, **init_params)

# Set the error difference to 0.5 (we are using NegativeLL)
minimizer.errordef = 0.5

# Fit for best parameters
mresult = minimizer.migrad()
mresult

The results for the Minuit minimizer is shown, with the fitted parameters for this 1 dataset as following:
* $\tau = 1.453 \pm 0.015$ (A.U.)
* $\Delta m = 19.93 \pm 0.08$ (A.U.)
* $ V = 0.099 \pm 0.014$ (A.U.)

Now, with the idea that we can have somewhat of a good fit on the parameters, now we would like to determine the expected precision on these fitted parameters. This is gonna be done by generating 100 toy datasets, each containing 10000 events like the dataset we generated above, and we will fit each one of them, store the fitted values and then obtain the precision and show the results.

First thing here is to start by creating a function which will be used to generate and fit datasets, and then store these fitted parameter values.

In [ ]:
def generate_fit_data(loBound, hiBound, nominal_params):
    # define a dictionary containing initial guess of params
    initial_params = {
        'tau'      : 1.1,
        'delta_m'  : 20.4,
        'V'        : 0.11,
    }

    # initialize pdf 
    pdf = DecayPDF(loBound, hiBound, *nominal_params.values())

    # Generate event in a loop (same as before)
    for i in range(nEvents):
        pdf.next()
    
    # Return the mass array and flush them so that it resets and the class can be used again for generating new toy datasets
    evts = pdf.returnMass()
    pdf.flushMass()

    # Now initialize the maximum likelyhood minimizing statistics
    fit_statistics = NegativeLLcalculator(pdf, evts)
    minuit_minimizer = Minuit(fit_statistics.evaluate, **initial_params)
    results = minuit_minimizer.migrad()

    # Now store the parameter values
    params_vals = []
    for i in initial_params.keys():
        params_vals.append(results.params[i].value)
    return params_vals    

Now with the above function, let's loop over 100 times to generate 100 toy dataset and fit them, each with 10000 events, and obtaining their optimized parameter values.

In [ ]:
# Define parameters again for completeness purpose
nominal_params_vals = {
    'tau'           : 1.5,
    'delta_m'       : 20.,
    'V'             : 0.1,
}

# Define a dictionary with empty lists for holding optimized params values
optimized_params_val_dict = {
    'tau_optim'           : [],
    'delta_m_optim'       : [],
    'V_optim'             : [],
}

# Now generate and fit 100 toy datasets
n_toy_datasets = 100
for i in range(n_toy_datasets):
    # Use our function for generating and fitting dataset
    optim_params_vals = generate_fit_data(loBound, hiBound, nominal_params_vals)

    # Then append the returned optimized parameter values to our dictionary
    for idx, key in enumerate(optimized_params_val_dict):
        optimized_params_val_dict[key].append(optim_params_vals[idx])

# Now our dict should contain the optimized parameters, now convert each list into np.array
for key in optimized_params_val_dict:
    optimized_params_val_dict[key] = np.array(optimized_params_val_dict[key])

With our optimized parameters from 100 toy datasets, the precision of our parameters can be calculated by taking the mean values of each list in the dictionary.

In [ ]:
# Calculate the precision of our parameters
tau_precision = optimized_params_val_dict['tau_optim'].mean()
delta_m_precision = optimized_params_val_dict['delta_m_optim'].mean()
V_precision = optimized_params_val_dict['V'].mean()

print('The tau parameter precision for our MC generated simulation is %.3f (A.U.).' % tau_precision)
print('The delta m parameter precision for our MC generated simulation is %.3f (A.U.).' % tau_precision)
print('The V parameter precision for our MC generated simulation is %.3f (A.U.).' % tau_precision)

Now, with the precision of our parameters explored and calculated, we would like to take a look at the bias for our parameters. The way we look at the bias here is by defining it to be the difference between the optimized parameter precision and the nominal truth parameter values.

In [ ]:
# calculate the bias on the parameters and print them
tau_bias = abs(nominal_params_vals['tau'] - tau_precision)
tau_bias = abs(nominal_params_vals['delta_m'] - delta_m_precision)
tau_bias = abs(nominal_params_vals['V'] - V_precision)

print('The tau parameter bias for our MC generated simulation is %.3f (A.U.).' % tau_bias)
print('The delta m parameter bias for our MC generated simulation is %.3f (A.U.).' % delta_m_bias)
print('The V parameter bias for our MC generated simulation is %.3f (A.U.).' % V_bias)

Next we need to calculate and assess the bias on the bias we calculated above for our optimized precision.